# 4.4 기울기  

만약 x0와 x1의 편미분을 함께 계산하고싶다면 어떻게 할까?  
모든 편미분을 묶어서 계산한다면, 이것을 벡터로 정리한 것을 기울기하고 한다. (gradient)   
기울기를 구현해보자

In [1]:
import numpy as np

def numerical_gradient(f, x):
    h = 1e-4 #0.0001
    grad = np.zeros_like(x) # x와 형상이 같은 배열을 생성, 0으로 가득 차 있음

    for idx in range(x.size):
        tmp_val = x[idx]
        
        # f(x+h) 계산
        x[idx] = tmp_val + h
        fxh1 = f(x)

        # f(x+h) 계산
        x[idx] = tmp_val - h
        fxh2 = f(x)

        grad[idx] = (fxh1 - fxh2) / (2*h)
        x[idx] = tmp_val

    return grad

In [2]:
# 실제 기울기 계산, (3,4), (0,2), (3,0)에서의 기울기를 구해보자

def function_2(x):
    return x[0]**2 + x[1]**2 #혹은 np.sum(x**2)로도 구현 가능

print(numerical_gradient(function_2, np.array([3.0,4.0])))
print(numerical_gradient(function_2, np.array([0.0,2.0])))
print(numerical_gradient(function_2, np.array([3.0,0.0])))

[6. 8.]
[0. 4.]
[6. 0.]


편미분의 결과에 마이너스를 붙인 벡터를 그려보면 다음과 같다.  

<img src = "./image/fig_4_9.png" width = "40%">  

그림을 보면 기울기는 함수의 최솟값을 가르키는 것 같다.  
사실 실제로 그렇지는 않고 각 지점에서 낮아지는 방향을 가리키는 것이다.  
즉 기울기가 가리키는 방향은 **각 장소에서 함수의 출력을 가장 줄이는 방향**이다.

## 4.4.1 경사법 (경사 하강법)  

기계학습 문제와 마찬가지로 신경망 역시 최적의 매개변수(가중치와 편향)를 학습 시에 찾아야 한다. 
여기서의 최적 == 손실함수가 최솟값이 되는 매개변수의 값  
그러나 이것을 알아내는 것은 만만치 않음  
여기서 기울기를 이용해서 함수의 최솟값 혹은 그에 준하는 작은 값을 찾으려는 것을 경사법이라고 한다.  

그러나 앞서 설명했듯이 기울기는 각 지점에서 함수의 값을 낮추는 방향이지 가르키는 방향에 정말 함수의 최솟값이 있는지는  
알 수가 없음  (특히 복잡한 함수에서는 없는 경우가 많음)  

```
함수가 극솟갑, 최솟값, 혹은 안장점이라는 곳에서 기울기가 0다.  
안장점은 어느 방향에서보면 극댓값이고 어느 방향에서보면 극솟값인 점이다.  
따라서 기울기가 0인 지점이 최솟값이라고 단정할 수는 없다.  
특히 복잡하고 찌그러진 모양의 함수는 평평한 곳으로 파고들면서 고원(plateau)이라고 하는,  
학습이 진행되지 않는 정체기에 빠질 수 있다.  
```

다만 기울어진 방향으로 가야만 함수의 값을 줄일 수 있기 때문에 일정거리만큼 이동하면서 기울기를 구하는 과정을 반복한다.  
이렇게 함수의 값을 조금씩 줄이는 것을 경사법이라고 한다.  
최솟값을 찾을때는 경사하강법, 최댓값을 찾을 때는 경사 상승법을 사용하는데 어짜피 부호만 다르다.  
다음은 경사법의 수식이다.  

<img src = "./image/e_4_7.png">  

η(eta): 갱신 량, 신경망에서는 학습률(Learning Rate)라고 부른다.  
    한번의 학습으로 얼마만큼 학습해야 할지, 매개변수의 값을 얼마나 갱신하는지를 나타낸 값  

계속해서 위의 식을 반복하면서 변수의 값을 줄여나가야 한다.  
또한 Learning Rate의 값은 0.001, 0.001 등 미리 정해 놔야 한다.  
적당한 값이 아닌경우 좋은 장소를 찾을 수 없다.  
신경망 학습은 이 값을 변경해 보면서 올바르게 학습 되고 있는지를 확인하며 진행한다.  
다음과 같이 경사하강법을 구현해 보았다.

In [3]:
def gradient_descent(f, init_x, lr = 0.01, step_num = 100):
    x = init_x

    for i in range(step_num):
        grad = numerical_gradient(f, x)
        x -= lr * grad
    return x

    - f: 최적화 함수  
    - int_x: 초깃값  
    - lr: learning rate  
    -step_num: 경사법에 따른 반복 횟수  
이 함수를 이용해 극솟값, 더 나아가 잘하면 최솟값까지 구하는 것이 가능하다.  

In [4]:
# 경사법으로 f(x0, x1) = x0^2+x1^2의 최솟값을 구해보자. 

init_x = np.array([-3.0, 4.0])
print(gradient_descent(function_2, init_x=init_x, lr=0.1, step_num=100))

[-6.11110793e-10  8.14814391e-10]


최종 결과는 [0,0]에 가까운 값이 되었다.  

<img src="./image/fig_4_10.png" width="40%">  

학습률이 너무 크거나 작으면 좋은 결과를 얻을 수 없다 했는데 실험해 보자.

In [5]:
# 학습률이 큰 경우: lr=10.0
init_x = np.array([-3.0, 4.0])
print(gradient_descent(function_2, init_x=init_x, lr=10.0, step_num=100))

# 학습률이 큰 경우: lr=1e-10
init_x = np.array([-3.0, 4.0])
print(gradient_descent(function_2, init_x=init_x, lr=1e-10, step_num=100))

[-2.58983747e+13 -1.29524862e+12]
[-2.99999994  3.99999992]


lr가 너무 높으면 큰값으로 발산, 너무 작으면 거의 갱신되지 않고 끝나버린다.  
이러한 lr과 같은 매개변수를 **하이퍼파라미터**라고 부른다.  
즉 기계가 학습을 통해 갱신되는 매개변수가 아닌 사람이 직접줘야 하는 매개변수를 뜻한다.  

## 4.4.2 신경망에서의 기울기  

신경망에서의 기울기: 가중치 매개변수에 대한 손실함수의 기울기  
가중치가 W, 손실함수가 L인 신경망에대한 경사는 다음과 같다.  

<img src = "./image/e_4_8.png" width = "20%">  

두번째 식의 각 원소는 각각의 원소에 관한 편미분이다.  
즉 가중치 w에 대해서 그 w를 조금 변경했을 때 손실함수 L이 얼마나 변화 하느냐를 나타낸다.  
따라서 가중치의 형태 (metrics)만큼에 대한 경사가 존재한다. (W와 형태가 같다.)  

In [10]:
import sys, os
sys.path.append(os.pardir)
from common.functions import softmax, cross_entropy_error
from common.gradient  import numerical_gradient

class simpleNet:
    def __init__(self):
        self.W = np.random.randn(2,3) # 정규분포로 초기화

    def predict(self, x):
        return np.dot(x, self.W)

    def loss(self, x, t):
        z = self.predict(x)
        y = softmax(z)
        loss = cross_entropy_error(y, t)

        return loss

복습해보면  

    - softmax: 출력층의 활성화 함수로 분류 문제에서 주로 사용됨  
    - cross_entry_error: 교차 엔트로피 오차 손실함수로 분류 문제에서 주로 사용됨  
    - 기울기를 구하는 편미분 함수  

In [56]:
net = simpleNet()
print(net.W)

x = np.array([0.6, 0.9])
p = net.predict(x)
print(p)

print("최대값 인덱스:",np.argmax(p)) # 최대값의 인덱스

t = np.array([0,0,1]) # 정답 레이블
print("손실 함수:",net.loss(x, t))

[[ 1.51959983 -0.10425575 -0.75642112]
 [-1.17692132  0.9014997   1.18206675]]
[-0.14746929  0.74879628  0.6100074 ]
최대값 인덱스: 1
손실 함수: 0.9623070755902571


In [59]:
def f(W): # W는 dummy 
    return net.loss(x, t)

dW = numerical_gradient(f, net.W)
print(dW)

[[ 0.10746287  0.26333077 -0.37079364]
 [ 0.1611943   0.39499616 -0.55619046]]


위 코드에서는 net.W를 인수로 받아 손실함수를 계산하는 새로운 함수 f를 정의  
그리고 편미분 함수에 넣어서 경사를 계산했다.  
dw의 값은 2 * 3의 배열로 이루어져있는데 여기서 h를 늘리면 dw * h만큼 손실함수의 값이 증가한다는 의미가 될 것이다.  
따라서 손실 함수를 줄인다는 측면 -> 양수인 dw의 값은 경사가 증가하므로 반대방향으로 진행시켜야 하고  
음수인 dw의 값은 경사가 감소하므로 가능 방향으로 진행 시키면 손실함수가 감소 할 것이다.  

신경망의 기울기를 구한 후에는 경사법에 따라 가중치 매개변수를 갱신하면 된다.  

참고로 위와같은 간단한 함수는 lanbda를 이용하는 것이 더 편하다.  

In [60]:
f = lambda w: net.loss(x,t)
dw = numerical_gradient(f,net.W)
dw

array([[ 0.10746287,  0.26333077, -0.37079364],
       [ 0.1611943 ,  0.39499616, -0.55619046]])